In [2]:
%load_ext autoreload
%autoreload 2

In [2]:
run_filters = True

In [3]:
import json
from preprocess_utils import save_jsonb, load_jsonb, filter_by_cz_lang, articles_num
from pathlib import Path

In [5]:

base_name = Path("articles_json")

In [5]:

base_name.mkdir(exist_ok=True)
if run_filters:
    for folder in Path("test_headline").iterdir():
        if not folder.is_dir():
            continue

        with open(base_name / f"{folder.name}.json", "w") as json_f:
            lines = []
            for file in folder.iterdir():
                if file.is_file():
                    with open(file, "r") as f:
                        jsoned = json.load(f)
                    lines.append(json.dumps(jsoned))
            save_jsonb(lines, json_f)
            


In [6]:
!conda install -y -c conda-forge fasttext

Solving environment: done

## Package Plan ##

  environment location: /home/kydliceh/.conda/envs/NLP

  added / updated specs:
    - fasttext


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1s             |       h7f8727e_0         3.6 MB
    ------------------------------------------------------------
                                           Total:         3.6 MB

The following packages will be UPDATED:

  openssl                                 1.1.1q-h7f8727e_0 --> 1.1.1s-h7f8727e_0



openssl-1.1.1s       | 3.6 MB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [7]:
!wget https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.ftz

--2022-11-08 14:15:57--  https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.ftz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 104.22.74.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 938013 (916K) [binary/octet-stream]
Saving to: ‘lid.176.ftz.1’

lid.176.ftz.1       100%[===================>] 916.03K  1.07MB/s    in 0.8s    

2022-11-08 14:15:59 (1.07 MB/s) - ‘lid.176.ftz.1’ saved [938013/938013]



In [9]:
print(list(articles_num(base_name)))

[('irozhlas.json', 200554), ('denik.json', 1006447), ('aktualne.json', 163792), ('idnes.json', 507828), ('novinky.json', 443767), ('ihned.json', 33672), ('seznamzpravy.json', 75204)]


In [6]:
cz_folder = Path("article_json_cz")

In [7]:
for file in base_name.iterdir():
    if file.is_file():
        save_jsonb(filter_by_cz_lang(file, 0.75, article_only=True), cz_folder / file.name)


NameError: name 'run_filters' is not defined

In [10]:
print(list(articles_num(cz_folder)))


[('seznamzpravy.json', 74519), ('ihned.json', 32393), ('denik.json', 980477), ('irozhlas.json', 199257), ('novinky.json', 421433), ('idnes.json', 491049), ('aktualne.json', 161096)]


12146
[('Milan Kopp', 10176), ('MF DNES', 9624), ('Milan Holakovský', 9487), ('Martin Hromádka', 7527), ('Marika Táborská', 6668), ('Markéta Kaclová', 6177), ('Stanislav Dvořák', 6041), ('Václav Havránek', 4805), ('Libor Kopl', 4647), ('Josef Kopecký', 4136), ('Štěpánka Saadouni', 3834), ('Mirka Spáčilová', 3684), ('Martin Ruščin', 3622), ('Daniel Seifert', 3552), ('Petr Turek', 3422), ('Alexandr Vanžura', 3323), ('Roman Brhel', 3256), ('Jiří Kohout', 3252), ('Daniela Tauberová', 3233), ('Kateřina Nič Husárová', 3200), ('Michal Hrabal', 3121), ('Martin Dorazín', 3053), ('Rudolf Muzika', 3045), ('Martin Vokurka', 3042), ('Jaroslav Špulák', 3021), ('Michal Burda', 2986), ('Tomáš Januszek', 2932), ('Iveta Nádvorníková', 2890), ('Iva Kovářová', 2866), ('Milan Kilián', 2822), ('Edwin Otta', 2820), ('Roman Cichocki', 2779), ('Daniela Loudová', 2779), ('Luboš Kurzweil', 2723), ('Petr Kinšt', 2712), ('Kamil Dubský', 2615), ('Petr Krňávek', 2552), ('Ivan Pavelek', 2550), ('František Kuba', 2504

In [ ]:
!conda install -y -c conda-forge ipywidgets ipython

Solving environment: done

## Package Plan ##

  environment location: /home/kydliceh/.conda/envs/NLP

  added / updated specs:
    - ipython
    - ipywidgets


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ipython-8.6.0              |     pyh41d4057_1         559 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         559 KB

The following packages will be UPDATED:

  ipython            pkgs/main/linux-64::ipython-8.4.0-py3~ --> conda-forge/noarch::ipython-8.6.0-pyh41d4057_1



ipython-8.6.0        | 559 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display

In [ ]:
from dataclasses import dataclass
possible_files = [None] + [x for x  in author_folder.iterdir() if x.is_file()]
modes = ["explore", "best_authors"]

class CachedData:
    def __init__(self, file: Path | None):
        self.file = file
        self.df = None
        if file != None:
            self.df = create_df(file)

    def update(self, file: Path):
        if file != self.file and file != None:
            self.df = create_df(file)

cached_df = CachedData(None)

def best_x_authors(file, num):
    auths = get_unique_authors(file)
    sorted_auths = sorted(auths.items(), key=lambda x: x[1], reverse=True)
    print(f"Total authors: {len(sorted_auths)}")
    print(f"Top {num} authors: ")
    for i in range(num):
        print(sorted_auths[i])


def interact_main(mode, file):
    if file == None:
        return

    if mode == "best_authors":
        best_x_authors(file, 100)

    if mode == "explore":
        cached_df.update(file)
        df = cached_df.df
        create_exploratory_plots(df)


interactive_plot = interactive(interact_main, mode=modes, file=possible_files)
display(interactive_plot)


interactive(children=(Dropdown(description='mode', options=('explore', 'best_authors'), value='explore'), Drop…

In [ ]:
min_length =show_outliers(create_df(author_folder / "seznamzpravy.json"), "article length", 50)

In [ ]:
show_df_lines(min_length, "seznamzpravy.json")

[{'author': ['Daniela Kučerová', 'Tomáš Trněný'], 'brief': 'Koronavirem se ve světě nakazilo více než 4,5 milionu lidí, z nichž 302 tisíc nákaze podlehlo. Přes 1,7 milionu lidí se uzdravilo. Nejhorší čísla dosud hlásí Spojené státy. Situaci ve světě sledujeme online.', 'category': 'Svět', 'comments_num': None, 'content': 'Vývoj dění ve světě jsme sledovali i včera:', 'headline': 'Brazílie má přes 200 000 nakažených, ministr zdravotnictví rezignoval', 'keywords': [], 'publication_date': '2020-05-15 04:33:06', 'url': 'https://www.seznamzpravy.cz/clanek/cina-mesic-nenahlasila-jedine-umrti-trump-si-s-ni-nechce-povidat-105899'}, {'author': ['Marek Vošahlík'], 'brief': 'Miliony lidí se přesunuly z kanceláří domů a síť čelí obrovskému zatížení. A těží z toho ty společnosti, které nabízejí cloudové služby, kancelářské balíky a komunikační nástroje - zkrátka jedním z velkých vítězů je Microsoft.', 'category': 'Byznys', 'comments_num': None, 'content': 'Microsoft hlásí rekordní čísla\nDaří se i 

In [ ]:
create_df(author_folder / "seznamzpravy.json").sort_values("article length", ascending=True).head(100)

,url,article length,headline length,brief length,num words,avg word length,num of non-alphanumeric,date,comments_num
1302,https://www.seznamzpravy.cz/clanek/budte-rozum...,7,65,220,2,3.000000,0,2021-02-15 13:40:31+00:00,None
18967,https://www.seznamzpravy.cz/clanek/matrjosky-n...,8,93,201,1,8.000000,0,2019-02-21 05:00:27+00:00,None
11527,https://www.seznamzpravy.cz/clanek/premier-tru...,10,91,406,2,4.500000,0,2019-09-19 13:39:07+00:00,None
13018,https://www.seznamzpravy.cz/clanek/nepruhledna...,11,68,199,2,5.000000,0,2020-05-18 15:15:20+00:00,None
29972,https://www.seznamzpravy.cz/clanek/ci-je-mount...,11,66,219,4,2.250000,1,2020-05-28 04:30:00+00:00,None
...,...,...,...,...,...,...,...,...,...
40862,https://www.seznamzpravy.cz/clanek/prohlasili-...,20,67,213,4,4.250000,0,2021-04-11 14:36:43+00:00,None
30261,https://www.seznamzpravy.cz/clanek/firma-ktera...,21,94,150,3,6.333333,0,2019-04-19 13:45:11+00:00,None
42640,https://www.seznamzpravy.cz/clanek/ceska-z-tym...,21,77,217,4,4.500000,0,2021-04-13 16:10:00+00:00,None
3781,https://www.seznamzpravy.cz/clanek/za-kulisy-p...,21,70,118,2,10.000000,0,2019-12-18 10:04:29+00:00,None


In [ ]:
final_file = Path("final.json")

In [ ]:
def add_server_name(js, filename: Path):
    js["server"] = filename.stem
    return js
if run_filters:
    final_file.unlink(missing_ok=True)
    for file in author_folder.iterdir():
        if file.is_file():
            loaded = load_jsonb(file)
            added_server = map(lambda x: add_server_name(x, file), loaded)
            save_jsonb(added_server, final_file, mode="a")

KeyboardInterrupt: 